In [2]:
#https://keras.io/examples/timeseries/timeseries_transformer_classification/
#https://saturncloud.io/blog/understanding-keras-conv1d-layer-parameters-filters-and-kernel-size/#:~:text=Filters%20in%20Conv1D&text=In%20the%20context%20of%20Conv1D,pattern%20in%20the%20input%20data.
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objects as go

df = pd.read_excel('Data2.xlsx', sheet_name=0)
ordered_df = df.iloc[::-1]

date_list = df.iloc[:, 0].values
price_list = df.iloc[:, 1].values
features_list = df.iloc[:, 1:16].values

#Scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features_list)
price_list = np.reshape(price_list, (price_list.shape[0], 1))
output_scaler = StandardScaler()
scaled_output_data = output_scaler.fit_transform(price_list)
#Train test split
training_data_len = math.ceil(len(scaled_data)* .85)
training_data_len

#Train test split
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
look_back_window = 20

for i in range(look_back_window, len(train_data)):
    x_train.append(train_data[i-look_back_window:i])
    y_train.append(train_data[i, 0])
#Convert to np array
x_train, y_train = np.array(x_train), np.array(y_train)

test_data = scaled_data[training_data_len-look_back_window:, :]

x_test = []
y_test = price_list[training_data_len:]
y_test = np.array(y_test)
y_test = np.reshape(y_test, (y_test.shape[0], 1))

for i in range(look_back_window, len(test_data)):
    x_test.append(test_data[i-look_back_window:i])

x_test = np.array(x_test)

In [3]:

from tensorflow import keras
from keras import layers
import keras_nlp
def build_model(
    intermediate_dim,
    input_shape, 
    num_heads,
    num_transformer_blocks,
    mlp_units,
    mlp_dropout=0,
    dropout = 0
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = keras_nlp.layers.TransformerEncoder(intermediate_dim=intermediate_dim, num_heads=num_heads,dropout=dropout, normalize_first=True)(x)

    te_input = x
    for _ in range(num_transformer_blocks):
        x = keras_nlp.layers.TransformerDecoder(intermediate_dim=intermediate_dim, num_heads=num_heads)(te_input, te_input)
    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

Using TensorFlow backend


In [90]:
print(np.sqrt(np.mean(y_test[0:-2] - y_test[1:-1])**2))

0.03247020585048753


In [92]:
input_shape = x_train.shape[1:]

model = build_model(
    64,
    input_shape,
    num_heads=4,
    num_transformer_blocks=6,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=8,
    callbacks=callbacks,
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()

Model: "model_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_42 (InputLayer)       [(None, 20, 14)]          0         
                                                                 
 transformer_encoder_153 (Tr  (None, 20, 14)           2648      
 ansformerEncoder)                                               
                                                                 
 transformer_encoder_154 (Tr  (None, 20, 14)           2648      
 ansformerEncoder)                                               
                                                                 
 transformer_encoder_155 (Tr  (None, 20, 14)           2648      
 ansformerEncoder)                                               
                                                                 
 transformer_encoder_156 (Tr  (None, 20, 14)           2648      
 ansformerEncoder)                                        

In [4]:
input_shape = x_train.shape[1:]

model = build_model(
    64,
    input_shape,
    num_heads=4,
    num_transformer_blocks=6,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=8,
    callbacks=callbacks,
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 14)]          0         
                                                                 
 transformer_encoder (Transf  (None, 20, 14)           2648      
 ormerEncoder)                                                   
                                                                 
 transformer_encoder_1 (Tran  (None, 20, 14)           2648      
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_2 (Tran  (None, 20, 14)           2648      
 sformerEncoder)                                                 
                                                                 
 transformer_encoder_3 (Tran  (None, 20, 14)           2648      
 sformerEncoder)                                             

ValueError: Found array with dim 3. None expected <= 2.

In [ ]:
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=512,
    num_heads=3,
    ff_dim=3,
    num_transformer_blocks=3,
    mlp_units=[256],
    mlp_dropout=0.2,
    dropout=0.2,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

In [ ]:
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()

In [9]:
decoder = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=64, num_heads=8)

# Create a simple model containing the decoder.
decoder_input = keras.Input(shape=(10, 64))
encoder_input = keras.Input(shape=(10, 64))
output = decoder(decoder_input, encoder_input)
model = keras.Model(
    inputs=(decoder_input, encoder_input),
    outputs=output,
)

# Call decoder on the inputs.
decoder_input_data = np.random.uniform(size=(2, 10, 64))
encoder_input_data = np.random.uniform(size=(2, 10, 64))
model.summary()
decoder_output = model((decoder_input_data, encoder_input_data))

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 10, 64)]     0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 10, 64)]     0           []                               
                                                                                                  
 transformer_decoder_2 (Transfo  (None, 10, 64)      41984       ['input_6[0][0]',                
 rmerDecoder)                                                     'input_7[0][0]']                
                                                                                                  
Total params: 41,984
Trainable params: 41,984
Non-trainable params: 0
______________________

In [7]:
print(decoder_output.shape)

(2, 10, 64)
